In [3]:
#from src.Crypto_Predictor import *
import src
import yfinance as yf
from datetime import date,timedelta
import keras
from keras.backend import clear_session
import pandas as pd

ModuleNotFoundError: No module named 'src'

In [9]:
coins = ['BTC-USD','ETH-USD','BNB-USD','SOL-USD','ADA-USD','XRP-USD','DOT-USD','AVAX-USD','LTC-USD','LINK-USD','ALGO-USD','MATIC-USD','UNI1-USD','EGLD-USD','TRX-USD','MANA-USD','SAND-USD','CAKE-USD','MKR-USD','SUSHI-USD','ICX-USD','EOS-USD','GAS-USD','LUNA1-USD']
coins# ICP WBTC AXS 

['BTC-USD',
 'ETH-USD',
 'BNB-USD',
 'SOL-USD',
 'ADA-USD',
 'XRP-USD',
 'DOT-USD',
 'AVAX-USD',
 'LTC-USD',
 'LINK-USD',
 'ALGO-USD',
 'MATIC-USD',
 'UNI1-USD',
 'EGLD-USD',
 'TRX-USD',
 'MANA-USD',
 'SAND-USD',
 'CAKE-USD',
 'MKR-USD',
 'SUSHI-USD',
 'ICX-USD',
 'EOS-USD',
 'GAS-USD',
 'LUNA1-USD']

In [7]:
def function(coins,Date,fd):
    for coin in coins:
        forecasts = {}
        for model_name in ['LSTM','BILSTM','CNN+LSTM','TRANSFORMER']:
            for hp in [20,50,100,365]:
                
                print(coin+'_'+model_name+'_'+str(fd)+'_'+str(hp))
                model = Crypto_Predictor(coin,model_name,add_technical=False,start_date=date(2015,1,1),end_date=Date-timedelta(days=1),train_test_split=1.0,history_points=hp,forecast_days=fd)
                model.run(plot=False)
                forecasts[coin+'_'+model_name+'_'+str(fd)+'_'+str(hp)] = model.forecast_price(Date,plot = False)
                clear_session()
            else:
                print(coin+'_'+model_name+'STACK'+'_'+str(fd)+'_'+str(20)+'_'+str(50)+'_'+str(100)+'_'+str(365))
                model1 = Crypto_Predictor(coin,model_name,add_technical=False,start_date=date(2015,1,1),end_date=Date-timedelta(days=1),train_test_split=1.0,history_points=20,forecast_days=fd)
                model2 = Crypto_Predictor(coin,model_name,add_technical=False,start_date=date(2015,1,1),end_date=Date-timedelta(days=1),train_test_split=1.0,history_points=50,forecast_days=fd)
                model3 = Crypto_Predictor(coin,model_name,add_technical=False,start_date=date(2015,1,1),end_date=Date-timedelta(days=1),train_test_split=1.0,history_points=100,forecast_days=fd)
                model4 = Crypto_Predictor(coin,model_name,add_technical=False,start_date=date(2015,1,1),end_date=Date-timedelta(days=1),train_test_split=1.0,history_points=365,forecast_days=fd)
                models = Crypto_Predictor(coin,'STACK',add_technical=False,start_date=date(2015,1,1),end_date=Date-timedelta(days=1),predictors=[model1,model2,model3,model4],forecast_days=fd)
                models.run(plot=False)
                forecasts[coin+'_'+model_name+'STACK'+'_'+str(fd)+'_'+str(20)+'_'+str(50)+'_'+str(100)+'_'+str(365)] = models.forecast_price(Date,plot = False)
                clear_session()

        for hp in [20,50,100,365]:
            print(coin+'_'+'MODEL'+'STACK'+'_'+str(fd)+'_'+str(hp))
            model1 = Crypto_Predictor(coin,'LSTM',add_technical=False,start_date=date(2015,1,1),end_date=Date-timedelta(days=1),train_test_split=1.0,history_points=hp,forecast_days=fd)
            model2 = Crypto_Predictor(coin,'BILSTM',add_technical=False,start_date=date(2015,1,1),end_date=Date-timedelta(days=1),train_test_split=1.0,history_points=hp,forecast_days=fd)
            model3 = Crypto_Predictor(coin,'CNN+LSTM',add_technical=False,start_date=date(2015,1,1),end_date=Date-timedelta(days=1),train_test_split=1.0,history_points=hp,forecast_days=fd)
            model4 = Crypto_Predictor(coin,'TRANSFORMER',add_technical=False,start_date=date(2015,1,1),end_date=Date-timedelta(days=1),train_test_split=1.0,history_points=hp,forecast_days=fd)
            models = Crypto_Predictor(coin,'STACK',add_technical=False,start_date=date(2015,1,1),end_date=Date-timedelta(days=1),predictors=[model1,model2,model3,model4],forecast_days=fd)
            models.run(plot=False)
            forecasts[coin+'_'+'MODEL'+'STACK'+'_'+str(fd)+'_'+str(hp)] = models.forecast_price(Date,plot = False)
            clear_session()
            
        f = pd.DataFrame(columns = ['Model_name','Model','History_Points','Forecast'])
        for i in forecasts.keys():
            a = i.split('_')
            b = {'Model_name':i,'Model':a[1],'History_Points':'_'.join(a[3:]),'Forecast':a[2]}
            for j,k in zip(forecasts[i]['Pred_Close'].index,forecasts[i]['Pred_Close'].values):
                b[j.date()]=k
            f=f.append(b,ignore_index=True)
        f = f.fillna('__')
        prices = yf.download(coin,Date+timedelta(days=1),Date+timedelta(days=fd))[['Close']].T
        diff = pd.DataFrame(f.iloc[:,4:].to_numpy()-prices.to_numpy(),columns = [i.strftime('%Y_%m_%d')+'_diff' for i in f.iloc[:,4:].columns])
        diff['5_Day_Average'] = diff.iloc[:,:5].sum(1)/5
        diff['10_Day_Average'] = diff.iloc[:,:10].sum(1)/10
        diff['15_Day_Average'] = diff.iloc[:,:15].sum(1)/15
        diff['30_Day_Average'] = diff.iloc[:,:30].sum(1)/30
        f = pd.concat([f,diff],axis=1)
        f.to_csv('results/'+coin+'_FORECAST_'+Date.strftime('%Y_%m_%d')+'.csv')
        rank = pd.concat([f.iloc[:,0],f.iloc[:,-4:].abs()],axis=1)
        df1 = rank.sort_values(by='5_Day_Average')[['Model_name','5_Day_Average']]
        df2 = rank.sort_values(by='10_Day_Average')[['Model_name','10_Day_Average']]
        df3 = rank.sort_values(by='15_Day_Average')[['Model_name','15_Day_Average']]
        df4 = rank.sort_values(by='30_Day_Average')[['Model_name','30_Day_Average']]
        df1.columns = ['Model_rank_5','5_Day_Average']
        df2.columns = ['Model_rank_10','10_Day_Average']
        df3.columns = ['Model_rank_15','15_Day_Average']
        df4.columns = ['Model_rank_30','30_Day_Average']
        df1.reset_index(drop=True,inplace=True)
        df5 = pd.concat([df1,df2.set_index(df1.index),df3.set_index(df1.index),df4.set_index(df1.index)],axis=1)
        df5.to_csv('results/'+coin+'_MODEL_RANKS_'+Date.strftime('%Y_%m_%d')+'.csv')
    return f,df5

In [8]:
f,df5 = function(coins,date(2021,12,18),10)

NameError: name 'coins' is not defined

In [10]:
f,df5 = function(coins,date(2021,11,28),10)

NameError: name 'date' is not defined

In [6]:
f,df5 = function(coins,date(2021,12,8),10)

SOL-USD_LSTM_10_20
-----------Creating Dataset-----------
[*********************100%***********************]  1 of 1 completed
-----------Dataset created-----------
-----------Creating Model-----------
-----------Model created-----------
-----------Training Model-----------
Epoch 00009: early stopping
-----------Model trained-----------
-----------Model Evaluation-----------
......Train RMSE...... ,  16.147127740306217
......Train MAPE...... ,  40.33177879941927
......Train MAE...... ,  7.920594348137578
......Train WAE...... ,  18.95836239238515
-----------Forecasting-----------
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
SOL-USD_LSTM_10_50
-----------Creating Dataset-----------
[*********************100%***********************]  1 of 1 completed
-----------Dataset created-----------
-----------Creating Model-----------
-----------Model created-----------
-----------Training Model-----------


In [7]:
f,df5 = function(coins,date(2021,11,20),30)

SOL-USD_LSTM_30_20
-----------Creating Dataset-----------
[*********************100%***********************]  1 of 1 completed
-----------Dataset created-----------
-----------Creating Model-----------
-----------Model created-----------
-----------Training Model-----------
Epoch 00006: early stopping
-----------Model trained-----------
-----------Model Evaluation-----------
......Train RMSE...... ,  23.363182676490897
......Train MAPE...... ,  117.49669495590169
......Train MAE...... ,  11.766333125984064
......Train WAE...... ,  35.357890028033744
-----------Forecasting-----------
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
SOL-USD_LSTM_30_50
-----------Creating Dataset-----------
[*********************100%***********************]  1 of 1 completed
-----------Dataset created-----------
-----------Creating Model-----------
-----------Model created-----------
-----------Training Model---------

In [1]:
import os
import xlsxwriter
import pandas as pd

In [2]:
files = [i for i in os.listdir('results/') if 'MODEL_RANKS' in i]
dfs = []
coinss = {}
for i in files:
    if i.split('_')[0] not in coinss:
        coinss[i.split('_')[0]] = []
    coinss[i.split('_')[0]].append(i)        
for coin in coinss:
    df3 = pd.DataFrame()
    for i in coinss[coin]:
        avg_name = ['Model_rank_10','10_Day_Average']
        df = pd.read_csv('results/'+i).loc[:2,avg_name]
        df1 = pd.DataFrame()
        for j in range(len(df)):
            df2 = pd.DataFrame(df.loc[j,:]).T
            df2.columns = ['Model_'+str(j+1),'Model_'+str(j+1)+'_avg']
            if len(df1)>0:
                df1 = pd.concat([df1, df2.set_index(df1.index)], axis=1)
            else:
                df1 = pd.concat([df1, df2], axis=1)
        df1.index = ['_'.join(i.split('_')[-3:])[:-4]]
        df3 = pd.concat([df3,df1])
    df3.name = coin
    dfs.append(df3)

In [3]:
writer = pd.ExcelWriter('Result.xlsx',engine='xlsxwriter')
workbook=writer.book
worksheet=workbook.add_worksheet('Result')
writer.sheets['Result'] = worksheet
start = 0
for i in range(len(dfs)):
    if i == 0:
        worksheet.write_string(start, 0, dfs[i].name)
    else:
        worksheet.write_string(start, 0, dfs[i].name)
    start+=1
    dfs[i].to_excel(writer,sheet_name='Result',startrow=start , startcol=0)
    start+=dfs[i-1].shape[0] + 4
writer.save()

In [4]:
files = [i for i in os.listdir('results/') if 'FORECAST' in i]
dfs = []
coinss = {}
for i in files:
    if i.split('_')[0] not in coinss:
        coinss[i.split('_')[0]] = []
    coinss[i.split('_')[0]].append(i)
for coin in coinss:
    df3 = pd.DataFrame()
    for i in coinss[coin]:
        df = pd.read_csv('results/'+i).iloc[:,1:]
        df['10_day_avg_pct_diff']=abs((df.iloc[:,4+df['Forecast'][0]:4+10+df['Forecast'][0]].to_numpy()/df.iloc[:,4:14].to_numpy()*100).sum(axis=1))/10
        df = df.sort_values(by='10_day_avg_pct_diff')[['Model_name','10_day_avg_pct_diff']].iloc[:3,:]
        df1 = pd.DataFrame()
        for j in range(len(df)):
            df2 = pd.DataFrame(df.iloc[j,:]).T
            df2.columns = ['Model_'+str(j+1),'Model_'+str(j+1)+'_avg_pct_diff']
            if len(df1)>0:
                df1 = pd.concat([df1, df2.set_index(df1.index)], axis=1)
            else:
                df1 = pd.concat([df1, df2], axis=1)
        df1.index = ['_'.join(i.split('_')[-3:])[:-4]]
        df3 = pd.concat([df3,df1])
    df3.name = coin
    dfs.append(df3)

In [5]:
writer = pd.ExcelWriter('Result1.xlsx',engine='xlsxwriter')
workbook=writer.book
worksheet=workbook.add_worksheet('Result')
writer.sheets['Result'] = worksheet
start = 0
for i in range(len(dfs)):
    if i == 0:
        worksheet.write_string(start, 0, dfs[i].name)
    else:
        worksheet.write_string(start, 0, dfs[i].name)
    start+=1
    dfs[i].to_excel(writer,sheet_name='Result',startrow=start , startcol=0)
    start+=dfs[i-1].shape[0] + 4
writer.save()